<a href="https://colab.research.google.com/github/Priya1881/NLP/blob/main/Topic_Modeling1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk import FreqDist
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import re
import spacy

import gensim
from gensim import corpora

# libraries for visualization
#import pyLDAvis
#import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Musical_Instruments_5.json', lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so ...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]","The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the sc...",5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]","The primary job of this device is to block the breath that would otherwise produce a popping sound, while allowing your voice to pass through with no noticeable reduction of volume or high frequen...",5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and prevents pops. Only thing is that the gooseneck is only marginally able to hold the screen in position and requires careful positioning of the clamp to avoi...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and performs like a studio filter. If you're recording vocals this will eliminate the pops that gets recorded when you sing.,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [ ]:
# function to plot most frequent terms
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()

  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

  # selecting top 20 most frequent words
  #d = words_df.nlargest(columns="count", n = terms) 
  #plt.figure(figsize=(20,5))
  #ax = sns.barplot(data=d, x= "word", y = "count")
  #ax.set(ylabel = 'Count')
  #plt.show()


freq_words(df['reviewText'])

In [ ]:
# remove unwanted characters, numbers and symbols
df['reviewText'] = df['reviewText'].str.replace("[^a-zA-Z#]", " ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(rev):
    rev_new = " ".join([i for i in rev if i not in stop_words])
    return rev_new

# remove short words (length < 3)
df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# remove stopwords from the text
reviews = [remove_stopwords(r.split()) for r in df['reviewText']]

# make entire text lowercase
reviews = [r.lower() for r in reviews]
freq_words(reviews, 35)

In [ ]:
!python -m spacy download en # one time run

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 47.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, tags=['NOUN', 'ADJ']): # filter noun and adjective
       output = []
       for sent in texts:
             doc = nlp(" ".join(sent)) 
             output.append([token.lemma_ for token in doc if token.pos_ in tags])
       return output

In [ ]:
tokenized_reviews = pd.Series(reviews).apply(lambda x: x.split())
print(tokenized_reviews[1])

['the', 'product', 'exactly', 'quite', 'affordable', 'realized', 'double', 'screened', 'arrived', 'even', 'better', 'expected', 'added', 'bonus', 'one', 'screens', 'carries', 'small', 'hint', 'smell', 'old', 'grape', 'candy', 'used', 'buy', 'reminiscent', 'sake', 'cannot', 'stop', 'putting', 'pop', 'filter', 'next', 'nose', 'smelling', 'recording', 'dif', 'needed', 'pop', 'filter', 'work', 'well', 'expensive', 'ones', 'may', 'even', 'come', 'pleasing', 'aroma', 'like', 'mine', 'buy', 'product']


In [ ]:
reviews_2 = lemmatization(tokenized_reviews)
print(reviews_2[1]) # print lemmatized review

['product', 'affordable', 'double', 'bonus', 'screen', 'small', 'hint', 'smell', 'old', 'grape', 'candy', 'reminiscent', 'sake', 'pop', 'filter', 'nose', 'recording', 'dif', 'pop', 'filter', 'work', 'expensive', 'one', 'pleasing', 'aroma', 'mine', 'product']


In [ ]:
reviews_3 = []
for i in range(len(reviews_2)):
    reviews_3.append(' '.join(reviews_2[i]))

df['reviews'] = reviews_3

freq_words(df['reviews'], 35)

In [ ]:
dictionary = corpora.Dictionary(reviews_2)

In [ ]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50)

In [ ]:
lda_model.show_topics()

[(0,
  '0.108*"string" + 0.045*"guitar" + 0.022*"capo" + 0.021*"good" + 0.020*"sound" + 0.018*"acoustic" + 0.018*"great" + 0.015*"tone" + 0.013*"electric" + 0.013*"time"'),
 (1,
  '0.053*"tuner" + 0.033*"pick" + 0.021*"tune" + 0.016*"easy" + 0.015*"great" + 0.013*"use" + 0.012*"guitar" + 0.012*"little" + 0.012*"light" + 0.010*"instrument"'),
 (2,
  '0.018*"sound" + 0.014*"bass" + 0.013*"head" + 0.011*"headphone" + 0.009*"low" + 0.009*"ear" + 0.008*"review" + 0.007*"high" + 0.007*"little" + 0.007*"way"'),
 (3,
  '0.055*"pedal" + 0.034*"amp" + 0.021*"sound" + 0.019*"tone" + 0.015*"effect" + 0.014*"great" + 0.014*"power" + 0.012*"good" + 0.011*"tube" + 0.010*"volume"'),
 (4,
  '0.045*"guitar" + 0.035*"strap" + 0.023*"case" + 0.014*"bag" + 0.012*"neck" + 0.011*"fit" + 0.011*"nice" + 0.011*"wave" + 0.011*"planet" + 0.009*"good"'),
 (5,
  '0.042*"good" + 0.033*"price" + 0.030*"cable" + 0.029*"great" + 0.026*"quality" + 0.023*"product" + 0.013*"cheap" + 0.013*"time" + 0.010*"guitar" + 0.010*"